# House Prices: Preprocessing and Modeling

In this Preprocessing and Modeling for the House Prices Case, I have built seven models whichlook for the mean squared error; in order to comapare which of the seven models have the lowest score. We take the lowest score because it provides us the highest accuracy of the predition between the actual and the predicited data. Models include Linear Regression, Ridge, Lasso, Mean Squred Error, Random Forest Regressor, Desicison Tree Regressor, and XG Boosting Regressor. 

# Imports 

In [34]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn import linear_model

# Load Data 

In [35]:
df = pd.read_csv('../House Prices Advance Regression Technique/train.csv')

# Preprocessing

We first remove the categorical features and account for the missing values. We then check on the DataFrame to check if everything is consistant. 

In [37]:
#Remove the categorical features 
dfTrain = df.select_dtypes(include = ['float64', 'int64'])

#fill in missing values  
dfTrain.drop(['LotFrontage', 'MasVnrArea', 'GarageYrBlt'],axis = 1, inplace = True)

dfTrain.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 35 columns):
 #   Column         Non-Null Count  Dtype
---  ------         --------------  -----
 0   Id             1460 non-null   int64
 1   MSSubClass     1460 non-null   int64
 2   LotArea        1460 non-null   int64
 3   OverallQual    1460 non-null   int64
 4   OverallCond    1460 non-null   int64
 5   YearBuilt      1460 non-null   int64
 6   YearRemodAdd   1460 non-null   int64
 7   BsmtFinSF1     1460 non-null   int64
 8   BsmtFinSF2     1460 non-null   int64
 9   BsmtUnfSF      1460 non-null   int64
 10  TotalBsmtSF    1460 non-null   int64
 11  1stFlrSF       1460 non-null   int64
 12  2ndFlrSF       1460 non-null   int64
 13  LowQualFinSF   1460 non-null   int64
 14  GrLivArea      1460 non-null   int64
 15  BsmtFullBath   1460 non-null   int64
 16  BsmtHalfBath   1460 non-null   int64
 17  FullBath       1460 non-null   int64
 18  HalfBath       1460 non-null   int64
 19  Bedroo

In [38]:
dfTrain['SalePrice']

0       208500
1       181500
2       223500
3       140000
4       250000
         ...  
1455    175000
1456    210000
1457    266500
1458    142125
1459    147500
Name: SalePrice, Length: 1460, dtype: int64

In [39]:
dfTrain.shape

(1460, 35)

## Convert Dataframe into dummies to include the categorical values 

Here we convert the categorical values into the numberical values with dummies, confirm the shape, and check the first five rows of the data.

In [40]:
dfTrain1 = pd.get_dummies(df)

In [41]:
dfTrain1.shape

(1460, 290)

In [42]:
dfTrain1.head()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,1,60,65.0,8450,7,5,2003,2003,196.0,706,...,0,0,0,1,0,0,0,0,1,0
1,2,20,80.0,9600,6,8,1976,1976,0.0,978,...,0,0,0,1,0,0,0,0,1,0
2,3,60,68.0,11250,7,5,2001,2002,162.0,486,...,0,0,0,1,0,0,0,0,1,0
3,4,70,60.0,9550,7,5,1915,1970,0.0,216,...,0,0,0,1,1,0,0,0,0,0
4,5,60,84.0,14260,8,5,2000,2000,350.0,655,...,0,0,0,1,0,0,0,0,1,0


In [43]:
dfTrain1.isna().sum().sum()

348

In [44]:
dfTrain1.fillna(0, inplace = True)

# Split and Train 

In [45]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(dfTrain1.drop(columns = ['SalePrice']),
                                                    dfTrain1.SalePrice,random_state =5)

In [46]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((1095, 289), (365, 289), (1095,), (365,))

# Model 1:  Linear Regression  

In [47]:
#r^2
lr = LinearRegression()
lr.fit(x_train,y_train)
print(r2_score(y_test,lr.predict(x_test)))

#mse
y_pred = lr.predict(x_test)
mean_squared_error(y_test,y_pred)


0.464440082802685


3313708396.8347783

# Model  2:  L2 Ridge 

In [30]:
#r^2
reg = linear_model.Ridge(alpha=.5)
reg.fit(x_train,y_train)
print(r2_score(y_test,reg.predict(x_test)))

#mse
y_pred = lr.predict(x_test)
mean_squared_error(y_test,y_pred)

0.8803196417692194


3313708396.8347783

# Model 3: L1 Lasso 

In [31]:
#r^2
regl2 = linear_model.Lasso(alpha=0.1)
regl2.fit(x_train,y_train)
print(r2_score(y_test,regl2.predict(x_test)))

#mse
y_pred = regl2.predict(x_test)
mean_squared_error(y_test,y_pred)

0.8844448631088188


/Users/aakash/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:532: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 224955238341.07874, tolerance: 694906333.7282932
  positive)


714982610.0085504

# Model 4 - Random Forest Regressor

In [20]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import cross_validate, GridSearchCV, learning_curve

In [21]:
RF_pipe = RandomForestRegressor(random_state=47)

In [22]:
rf1 = RF_pipe.fit(x_train, y_train)
ypred = rf1.predict(x_test)

In [23]:
mean_squared_error(y_test,ypred)

761948359.2807658

# Model 5 - Decision Tree Regressor 

In [24]:
from sklearn.tree import DecisionTreeRegressor
from sklearn import tree, metrics

In [25]:
reg = DecisionTreeRegressor()
reg.fit(x_train, y_train)

DecisionTreeRegressor()

In [26]:
ypred = reg.predict(x_test)

In [27]:
mean_squared_error(y_test,ypred)

1698292057.7123287

# Model 6 - XG Boosting Regressor

In [28]:
from sklearn.ensemble import GradientBoostingRegressor

reg = GradientBoostingRegressor()
reg.fit(x_train, y_train)

GradientBoostingRegressor()

In [29]:
ypred = reg.predict(x_test)
mean_squared_error(y_test,ypred)

618215953.5030107

According to our models, we best see that the Linear Regression is the best model to choose becasue it illistrates that the mena square error score of Model 6:618215953.5030107 is our best accuracy of the predition between the actual and the predicited data.